In [29]:
# Bibliotecas
import pandas as pd
import numpy as np
from numpy import sqrt
from numpy import vstack
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, classification_report
from sklearn.metrics import roc_curve
from tqdm.notebook import trange, tqdm
import time
import torch
from torch import nn
from torch.nn.functional import normalize
from torchvision import datasets, transforms
import torch.utils.data as data
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch import optim
from torch.optim import SGD
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from torch.nn.modules.activation import ReLU
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from datetime import datetime

In [30]:
torch.randn(5).cuda()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
# Train, Val split
roughData = pd.read_csv('seattleWeather_1948-2017.csv')
roughData

In [ ]:
# NaN values deletion
roughData.dropna(inplace=True)
roughData.reset_index(inplace=True, drop=True)
roughData

In [31]:
#Create a n array with the values of PRCP, TMAX, TMIN, and RAIN
roughData = roughData[['PRCP', 'TMAX', 'TMIN', 'RAIN']]
roughData

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [32]:
#Eliminate possible void values
roughData = roughData[roughData['PRCP'] != 'T']
roughData = roughData[roughData['TMAX'] != 'T']
roughData = roughData[roughData['TMIN'] != 'T']
roughData = roughData[roughData['RAIN'] != 'T']
roughData

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
#Convert the values of PRCP, TMAX, TMIN, and RAIN to float
roughData['PRCP'] = roughData['PRCP'].astype(float)
roughData['TMAX'] = roughData['TMAX'].astype(float)
roughData['TMIN'] = roughData['TMIN'].astype(float)
roughData['RAIN'] = roughData['RAIN'].astype(float)
roughData

In [ ]:
#Convert the dataframe to a numpy array
roughData = roughData.to_numpy()


In [33]:
#Create the dataset class using the PRCP, TMAX, TMIN as inputs and RAIN as label
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.len = data.shape[0]
        self.x_data = torch.from_numpy(self.data[:, :3])
        self.y_data = torch.from_numpy(self.data[:, 3])
        self.y_data = self.y_data.type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


In [34]:
#Create the train and test dataset using the Dataset class and the data from the dataframe with 90% of the data for training and 10% for testing
train_data = Dataset(roughData[:int(0.9 * len(roughData))])
test_data = Dataset(roughData[int(0.9 * len(roughData)):])

In [35]:
#Create the train and test dataloader using the train and test dataset
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=True)


In [36]:
# Definir modelo

class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(3, 3)
        self.l2 = nn.Linear(3, 3)
        self.l3 = nn.Linear(3, 1)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        h1 = nn.functional.relu(self.l1(x))  # Hidden layer
        h2 = nn.functional.relu(self.l2(h1))
        do = self.do(h2 + h1)
        logits = self.l3(do)
        return logits



In [37]:
model = ResNet()

In [38]:
# Definir optimizador
params = model.parameters()
optimizer = optim.SGD(params, lr=1e-2)

In [ ]:
# Definir función de costo
loss = nn.CrossEntropyLoss()


64

In [40]:
# Ciclos de entrenamiento y validacion
nb_epochs = 25
for epoch in range(nb_epochs):
    losses = list()
    accuracies = list()
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        x, y = data, target

        # x : b x 1 x 4 x 4
        b = x.size(0)
        x = x.view(b, -1)

        # 1. forward
        l = model(x.float())  # l: logits

        # 2. evaluación de la función objetivo
        j = loss(l, y)

        # 3. limpieza de gradientes
        model.zero_grad()

        # 4. Backpropagation: acumular derivadas parciales de la función de costo
        j.backward()

        # 5. retroceder hacia la dirección opuesta del gradiente (aprendizaje)
        optimizer.step()

        losses.append(j.item())
        accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())

    print(f'Epoch {epoch + 1}', end=', ')
    print(f'Training loss: {torch.tensor(losses).mean():.2f}', end=', ')
    print(f'Training accuracy: {torch.tensor(accuracies).mean():.2f}')

    losses = list()
    accuracies = list()
    model.eval()
    for batch_idx, (data, target) in enumerate(test_loader):
        x, y = data, target

        # x : b x 1 x 4 x 4
        b = x.size(0)
        x = x.view(b, -1)

        # 1. forward
        with torch.no_grad():
            l = model(x.float())  # l: logits

        # 2. evaluación de la función objetivo
        j = loss(l, y)

        losses.append(j.item())
        accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())

    print(f'Epoch {epoch + 1}', end=', ')
    print(f'Validation loss: {torch.tensor(losses).mean():.2f}', end=', ')
    print(f'Validation accuracy: {torch.tensor(accuracies).mean():.2f}')


IndexError: Target 1 is out of bounds.

64